
Καμάρης Άγγελος
sdi1900070

**Τεχνητή Νοημοσύνη ΙΙ**

(ακαδ. έτος 2022-23)

Άσκηση 2 (δημιουργία βάσης):



In [2]:
!pip install sparqlwrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import time
from urllib.error import HTTPError
import pandas as pd
from google.colab import drive

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

#function to make api request
def query_sparql(query):
    try:
        sparql.setQuery(query)
        results = sparql.query().convert()
        return results
    except HTTPError as e:
        if e.code == 429: # HTTP 429 = Too Many Requests
            print("Too many requests. Sleeping for 10 seconds...")
            time.sleep(10)
            return query_sparql(query)
        else:
            raise e

#Function to create span for our dataframe
def create_span(entity_label, question):
              words = question.lower().split()
              span = []
              entity_words = entity_label.lower().split()
              span = [0] * len(words)
              for e_word in entity_words:
                count=0
                for word in words:
                    if e_word in word:
                        span[count]=1
                    count+=1
              return span



drive.mount('/content/drive')


def create_data(filename):
  dataframe=[]
  relation_vocabulary=[]
  entity_map=[]
  count=1
  i=0
  # Open the input file from your Google Drive
  with open(filename, 'r') as f:
      # Loop through the lines in the file
      for line in f:
          i+=1
          dataset={}
          flag=0
          # Split the line into its components
          entity_id, relation_id, target_id, question = line.split('\t')
          

          # Make sure relations start with P, R doesn't work
          if(relation_id[0]=='R'):
            relation_id='P'+ relation_id[1:]

          if(i%100==0):
            print(i)

          #query that returns the labels for our entity and relation
          query="""
            SELECT ?entity_label ?relation_label WHERE {
                {
                    wd:%s rdfs:label ?entity_label .
                    FILTER (lang(?entity_label) = "en")
                }
                UNION
                {
                    wd:%s rdfs:label ?relation_label .
                    FILTER (lang(?relation_label) = "en")
                }
            }
            """% (entity_id, relation_id)
          results = query_sparql(query)
          
          #get results info
          response = results['results']['bindings']
          #make sure both entity id and relation id return labels, or else it is unusable
          if len(response)==2:
            #get and store the info in our dataset
            if 'entity_label' in response[0]:
              dataset['id']=i
              dataset['entity_id']=entity_id
              dataset['entity_label']=response[0]['entity_label']['value']
              dataset['relation_id']=relation_id
              dataset['relation_label']=response[1]['relation_label']['value']
            else:
                dataset['id']=i
                dataset['entity_id']=entity_id
                dataset['entity_label']=response[1]['entity_label']['value']
                dataset['relation_id']=relation_id
                dataset['relation_label']=response[0]['relation_label']['value']
            dataset['question']=question.rstrip('\n')
            #create and store span
            dataset['span'] = create_span(dataset['entity_label'], dataset['question'])
            dataframe.append(dataset)
            relation_vocabulary.append(dataset['relation_id'])
            entity_map.append({dataset['entity_label']:dataset['entity_id']})
  return dataframe, relation_vocabulary, entity_map

traindata, rvtrain, emtrain=create_data('/content/drive/My Drive/annotated_wd_data_train_answerable.txt')
testdata, rvtest, emtest=create_data('/content/drive/My Drive/annotated_wd_data_test_answerable.txt')
valdata, rvval, emval=create_data('/content/drive/My Drive/annotated_wd_data_valid_answerable.txt')
traindf = pd.DataFrame(traindata)
print(traindf.iloc[:10, :])
testdf = pd.DataFrame(testdata)
print(testdf.iloc[:10, :])
valdf = pd.DataFrame(valdata)
print(valdf.iloc[:10, :])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.5 MB/s eta 0:00:00
Mounted at /content/drive
100
200
300
400
500
600
Too many requests. Sleeping for 10 seconds...
Too many requests. Sleeping for 10 seconds...
700
Too many requests. Sleeping for 10 seconds...
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400


In [3]:
traindf.to_csv('my_train.csv', index=False)
testdf.to_csv('my_test.csv', index=False)
valdf.to_csv('my_val.csv', index=False)